In [ ]:
!pip install pytube
!pip install moviepy
!pip install SpeechRecognition
!pip install python-docx
!pip install autocorrect
!pip install pytube
!pip install tensorflow
!pip install pydub
!pip install autocorrect
!pip install google-cloud-speech

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=abdd0eb106ff229b1416c7a685fa9a66753fb62ce6784ce23c6ee32d453f9876
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.8/284.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
import moviepy.editor as mp
import speech_recognition as sr
from docx import Document
from autocorrect import Speller
from pytube import YouTube
import time
import speech_recognition as sr
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
from google.cloud import speech
from docx import Document
from autocorrect import Speller
from collections import Counter

In [ ]:
def download_video(youtube_url):
    try:
        yt = YouTube(youtube_url)
        # Get the highest resolution video
        video = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
        video_path = video.download()
        return video_path
    except Exception as e:
        print("Error:", e)
        return None

def extract_audio(video_path):
    # Load video and extract audio
    video = mp.VideoFileClip(video_path)
    audio_path = "audio_extracted.wav"
    video.audio.write_audiofile(audio_path)
    return audio_path

In [ ]:
def transcribe_audio(audio_path):
    # Initialize the recognizer
    recognizer = sr.Recognizer()

    # Load the audio file
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)

    # Transcribe audio using Google Web Speech API
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        print("Google Web Speech API could not understand audio")
        return ""
    except sr.RequestError as e:
        print(f"Error with Google Web Speech API: {e}")
        return ""
def correct_spelling(text):
    spell = Speller(lang='en')
    corrected_text = spell(text)
    return corrected_text


In [ ]:
def generate_notes(transcribed_text, heading):
    # Split transcribed text into words
    words = transcribed_text.split()

    # Auto-correct spelling in words
    corrected_words = [correct_spelling(word) for word in words]

    # Reduce repeated words
    unique_words = set(corrected_words)

    # Generate notes as a list of sentences
    notes = ' '.join(unique_words)

    # Add heading to the notes
    notes_with_heading = f"{heading}\n\n{notes}"

    return notes_with_heading

In [ ]:
def save_notes_to_doc(notes, output_path):
    doc = Document()
    doc.add_paragraph(notes)
    doc.save(output_path)

def browse_path_or_url():
    while True:
        user_input = input("Enter the path to the video file or the YouTube video URL (or 'exit' to quit): ")
        if user_input.lower() == "exit":
            return None
        elif os.path.isfile(user_input):
            return user_input
        elif "youtube.com" in user_input or "youtu.be" in user_input:
            return user_input
        else:
            print("Invalid input. Please enter a valid file path or YouTube video URL.")

def enter_heading():
    heading = input("Enter the heading for the notes: ")
    return heading

In [ ]:

if __name__ == "__main__":
    # Prompt user to select a video file or enter a YouTube URL
    print("Please enter the path to the video file or the YouTube video URL.")
    print("Type 'exit' to quit.")
    video_input = browse_path_or_url()

    if not video_input:
        print("No input provided. Exiting.")
        exit()

    if "youtube.com" in video_input or "youtu.be" in video_input:
        # Download the YouTube video
        print("Downloading the video...")
        video_path = download_video(video_input)
        if not video_path:
            print("Failed to download the video. Exiting.")
            exit()
    else:
        video_path = video_input

    # Prompt user to enter a heading for the notes
    heading = enter_heading()

    output_path_audio = "audio_extracted.wav"
    output_path_doc = "notes.docx"

    # Extract audio from the video
    audio_path = extract_audio(video_path)

    # Transcribe the audio using Google Web Speech API
    transcribed_text = transcribe_audio(audio_path)

    # Generate notes with auto-correction and reduced repeated words, including the specified heading
    notes = generate_notes(transcribed_text, heading)

    # Save the notes to a Word document
    save_notes_to_doc(notes, output_path_doc)

    print(f"Notes generated and saved to '{output_path_doc}'.")